In [ ]:
!pip install -q Pillow

In [ ]:
import os
from PIL import Image
import numpy as np
import tensorflow as tf

## Data Gathering

Download dataset and extract to `train_data` and `val_data`.

In [ ]:
!wget http://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
!wget http://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip

--2020-12-26 18:53:26--  http://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.79.128, 108.177.96.128, 108.177.119.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.79.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip’

horse-or-human.zip  100%[===================>] 142.65M  68.7MB/s    in 2.1s    

2020-12-26 18:53:29 (68.7 MB/s) - ‘horse-or-human.zip’ saved [149574867/149574867]

--2020-12-26 18:53:29--  http://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.218.128, 74.125.128.128, 74.125.143.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.218.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M

In [ ]:
TRAIN_DIR = "./train_data"
VAL_DIR = "./val_data"
!unzip -q horse-or-human.zip -d $TRAIN_DIR
!unzip -q validation-horse-or-human.zip -d $VAL_DIR

Read a sample image and check pixel range.

In [ ]:
train_sample_fn = os.listdir(os.path.join(TRAIN_DIR, "horses"))[0]
train_sample_path = os.path.join(TRAIN_DIR, "horses", train_sample_fn)

**Note:** The last channel is transparency.

In [ ]:
img = Image.open(train_sample_path)
img_data = np.asarray(img)

print("Image shape:", img_data.shape)
print("Pixel in range:", np.min(img_data), np.max(img_data))

print(img_data[:1])

Image shape: (300, 300, 4)
Pixel in range: 1 255
[[[100 108 123 255]
  [100 108 123 255]
  [ 99 108 123 255]
  ...
  [159 166 180 255]
  [163 171 185 255]
  [169 178 191 255]]]


## Data Flow

In [ ]:
img_size = (300, 300)
img_shape = (*img_size, 3)    # We didn't include transparency channel

Create data flow from a directory, each image is scaled to the range 0-255.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def gen_new_data(data_folder):
  data_gen = ImageDataGenerator(rescale=1./255)
  data = data_gen.flow_from_directory(
      data_folder,
      target_size=img_size,
      batch_size=64,
      class_mode="binary",
  )
  return data

In [ ]:
train_data_gen = gen_new_data(TRAIN_DIR)
val_data_gen = gen_new_data(VAL_DIR)

## Load Pretrained Model

Load InceptionV3 pretrained on ImageNet and freeze all layers above `mixed7`. This layer will give us features extracted from InceptionV3. We're gonna replace the top of the model with our own classifier.

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
pretrained_model = InceptionV3(include_top=False, weights="imagenet", input_shape=(150, 150, 3))

In [ ]:
# Freeze all layers
for layer in pretrained_model.layers:
  layer.trainable = False

# We keep all layers up to `mixed7`
last_pretrained_layer = pretrained_model.get_layer(name="mixed7")
print("Last layer shape:", last_pretrained_layer.output_shape)

## Model Architecture & Training

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

In [ ]:
train_model = Flatten()(last_pretrained_layer.output)
train_model = Dense(1024, activation="swish")(train_model)
train_model = Dropout(0.1)(train_model)
train_model = Dense(1, activation="sigmoid")(train_model)

In [ ]:
model = Model(pretrained_model.input, train_model)
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-4)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["acc"])
model.fit(train_data_gen, epochs=5, validation_data=val_data_gen)

In [ ]:
model.evaluate(val_data_gen)

In [ ]:
    model.save("mymodel.h5")